# Nettoyage des données EXIOBASE3

Nettoyage des données d'EXIOBASE3 pour obtenir les DataFrames correspondant aux niveaux d'émissions de gaz à effet de serre (GES) émis par chaque région et destinés à la consommation finale.

In [2]:
#importer pymrio et pandas
import pymrio
import pandas as pd

In [3]:
#Fonction load qui prend 'year' comme argument
def load(year):
    #Chargement des donnée EXIOBASE3 pour chaque année 
    path = f"C:/Master 1/Stage/EXIOBASE3/IOT_{year}_ixi.zip"
    exio3 = pymrio.parse_exiobase3(path=path)

    #Diagonalisation de l'impact associé aux emissions de GES
    diag_GHG = exio3.impacts.diag_stressor('GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 1999) | GWP100 (IPCC, 2007)')

    #Les valeurs de la diagonalisation sont attribués à 'exio3.GHG_source'
    exio3.GHG_source = diag_GHG

    #Commande pymrio qui permet de compléter les parties manquantes de EXIOBASE3
    exio3.calc_all()

    #Agrège les émissions de GES par région pour obtenir les émissions associées à la consommation finale
    GHG_source_reg = exio3.GHG_source.D_cba.groupby(
    level='region', axis=0).sum().groupby(
    level='region', axis=1).sum()
    
    #Supprimer les lignes où les valeurs sont NaN
    df = GHG_source_reg.dropna(how='all', axis=0)

    #Formation de liste pour former de nouvelles régions
    UE = ['AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 
                'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
    MO = ['WM', 'CY']
    Europe = ['WE', 'CH', 'TR', 'GB']
    Asie_Pacifique = ['WA', 'TW', 'KR']
    Afrique = ['WF', 'ZA']

    #Création de nouvelles colonnes pour chaque nouvelle région
    df['UE'] = df[UE].sum(axis=1)
    df['Moyen-Orient'] = df[MO].sum(axis=1)
    df['Europe'] = df[Europe].sum(axis=1)
    df['Asie Pacifique'] = df[Asie_Pacifique].sum(axis=1)
    df['Afrique'] = df[Afrique].sum(axis=1)

    #Création de nouvelles lignes pour chaque nouvelle région
    df.loc['UE'] = df.loc[UE].sum()
    df.loc['Moyen-Orient'] = df.loc[MO].sum()
    df.loc['Europe'] = df.loc[Europe].sum()
    df.loc['Asie Pacifique'] = df.loc[Asie_Pacifique].sum()
    df.loc['Afrique'] = df.loc[Afrique].sum()

    #Supprime les lignes et colonnes des pays individuels inclus dans les nouvelles régions créées.
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=0)
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=1)

    #Réorganise l'ordre des pays de la DataFrame pour les lignes et les colonnes
    order = ["CN", "US", "Asie Pacifique", "IN", "UE",  "Afrique", 
               "Moyen-Orient", "RU", "Europe", "WL", "JP",  "ID",
               "BR", "MX", "AU",  "CA"]
    df = pd.DataFrame(df, index=order)
    df = df[order]

    #Renomme le noms des lignes et des colonnes pour rendre la DataFrame plus compréhensible
    region = ["Chine", "Etats-Unis", "Asie Pacifique", "Inde", "Union européenne", "Afrique", "Moyen-Orient", "Russie", "Europe",  "Amerique Latine et Caraïbes", "Japon",  "Indonésie", "Brésil","Mexique",  "Australie", "Canada" ]
    df.columns = region
    df.index = region
    
    #Résult de la fonction load
    return df

#### Étapes du nettoyage: <br>
1. Choisir le stresseur ou l'impact qui nous intéresse pour notre analyse. Ici nous utilisons l'impact lié aux émissions de GES. 
2. Diagonalisation de ce même stresseur/impact. Dans ce cas, cela permet d'exprimer les émissions de GES de chaque secteur et chaque région. 
3. Agrèger ce stresseur/impact par région pour obtenir le niveau du stresseur/impact associé à la consommation finale pour chaque région. Dans ce cas, cela permet d'agréger les émissions de GES issues de tous les secteurs confondus.
4. Réduire le nombre de régions pour réduire la taille de la DataFrame afin d'être plus visible lorsqu'on voudra la représenter visuellement. Pour cela, nous gardons certaines régions telles qu'elles sont définies dans EXIOBASE3, comme la Chine, des Etats-Unis, de l'Inde, de la Russie, de l'Indonésie, du Japon, du Mexique, du Brésil et du Canada. Cependant, nous regroupons:
- Tous les pays de l'Union européenne dans la région "Union européenne".
- Tous les pays européens qui ne font pas partie de l'Union européenne dans la région 'Europe'.
- Tous les pays du Moyen-Orient (hors la Turquie qui est déjà comprise dans la région "Europe) dans la région 'Moyen-Orient'.
- Tous les pays d'Asie-Pacifique (hors le Japon, la Chine, l'Inde et l'Indonésie) dans la région 'Asie-Pacifique'.
- Tous les pays d'Afrique (hors l'Egypte qui est déjà comptabilisée dans la région 'Moyen-Orient) dans la région 'Afrique'.
Cela permet de réduire le nombre de région dans notre DataFrame de 49 régions à 16 régions. <br>

La liste détaillée des pays regroupés dans les 49 régions des données EXIOBASE3 peut être trouvée à ce [lien](https://github.com/MarDrd/exiobase3-manipulation/blob/main/R%C3%A9gions.ipynb).

La liste détaillée des pays regroupés dans les 16 nouvelles régions peut être trouvée à ce [lien](https://github.com/MarDrd/exiobase3-manipulation/blob/main/Impact%20-%20Consommation%20Finale/R%C3%A9gion%20Description/Composition_Regions.ipynb).

In [4]:
#Chargement de la base de données nettoyée de 2022
df_2022 = load(2022)

c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Use

In [5]:
#Enregistrement de la base de donnée en format csv
df_2022.to_csv('df_GHG_2022')

Les données de chaque année d'EXIOBASE3 étant lourdes, je déconseille de charger plusieurs années à la fois et de plutôt les charger une par une.

Le chargement étant long, vous pouvez retrouver toutes les bases de données néttoyés représentant les émissions de GES liées à la consommation finale de 1995 à 2022 en format CSV en suivant ce [lien](https://github.com/MarDrd/exiobase3-manipulation/tree/main/Impact%20-%20Consommation%20Finale/GES%20-%20Consommation%20Finale/GES%20-%20final%20conso). Cela permet un téléchargement beaucoup plus rapide de ces mêmes bases de données.

In [6]:
#Chargement de la base de donnée
df = pd.read_csv('C:\Master 1\exiobase3-manipulation\Impact - Consommation Finale\GES - Consommation Finale\GES - final conso\df_GHG_2022.csv')

In [7]:
#Afficher la base de donnée 2022
df

,Chine,Etats-Unis,Asie Pacifique,Inde,Union européenne,Afrique,Moyen-Orient,Russie,Europe,Amerique Latine et Caraïbes,Japon,Indonésie,Brésil,Mexique,Australie,Canada
Chine,1.172497e+13,4.226491e+11,6.378631e+11,1.029587e+11,2.542195e+11,9.891715e+10,1.584878e+11,3.853897e+10,8.531757e+10,9.685833e+10,1.168426e+11,3.421645e+10,3.907341e+10,2.454217e+10,3.130982e+10,4.165509e+10
Etats-Unis,8.750271e+10,4.357353e+12,9.236064e+10,2.379245e+10,9.710983e+10,1.740554e+10,3.334756e+10,5.194852e+09,3.376874e+10,7.409290e+10,4.031793e+10,4.674862e+09,2.010811e+10,6.397212e+10,6.391105e+09,7.038269e+10
Asie Pacifique,5.928809e+11,2.904280e+11,2.530916e+12,1.135787e+11,3.269152e+11,9.900515e+10,1.129265e+11,5.274800e+10,1.137412e+11,4.899165e+10,1.461975e+11,8.031297e+10,1.824609e+10,1.381192e+10,7.411451e+10,2.617918e+10
Inde,5.281001e+10,8.517561e+10,1.324073e+11,3.613719e+12,8.245074e+10,4.434384e+10,1.083409e+11,6.858290e+09,3.104210e+10,1.488577e+10,1.236043e+10,1.132690e+10,6.990345e+09,5.100568e+09,6.596916e+09,7.268872e+09
Union européenne,9.885223e+10,1.368905e+11,9.080989e+10,2.429518e+10,2.916673e+12,6.484887e+10,7.504863e+10,3.454856e+10,2.079430e+11,2.889655e+10,2.503281e+10,6.152704e+09,1.768632e+10,1.118301e+10,1.184784e+10,1.952101e+10
Afrique,9.892487e+10,7.144618e+10,6.701050e+10,6.191249e+10,1.919576e+11,2.210419e+12,7.994482e+10,8.026747e+09,4.268475e+10,1.216868e+10,1.213066e+10,7.103954e+09,1.234085e+10,4.860922e+09,6.342286e+09,7.227786e+09
Moyen-Orient,2.422756e+11,1.653195e+11,2.105309e+11,1.267189e+11,1.708965e+11,8.729435e+10,1.595254e+12,1.433141e+10,6.707269e+10,1.974455e+10,8.928854e+10,1.708492e+10,1.220731e+10,7.790333e+09,1.549409e+10,1.230463e+10
Russie,1.620748e+11,5.626536e+10,1.454480e+11,2.309255e+10,2.962227e+11,3.183961e+10,4.161487e+10,1.355985e+12,1.527611e+11,1.616756e+10,2.574908e+10,5.040296e+09,6.954969e+09,4.285300e+09,4.448559e+09,6.192348e+09
Europe,3.548059e+10,5.287426e+10,4.573020e+10,1.178005e+10,1.869117e+11,3.724665e+10,6.681677e+10,4.677245e+10,1.062805e+12,1.230974e+10,8.354257e+09,3.637459e+09,5.057685e+09,3.137437e+09,5.330506e+09,7.045756e+09
Amerique Latine et Caraïbes,1.240386e+11,1.227821e+11,4.525974e+10,2.229248e+10,6.624495e+10,1.601574e+10,2.653906e+10,1.835639e+10,3.270084e+10,8.761539e+11,1.083279e+10,2.615813e+09,3.391146e+10,1.237366e+10,2.323305e+09,8.342483e+09
